In [1]:
import sys
import sys
sys.path.append('../../')
from omnicell.constants import *

from omnicell.models.gears.gears import GEARS
import omnicell.models.gears.pertdata as pertdata_new
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.pertdata import PertData
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.gears import GEARS
import scanpy as sc
import torch
import numpy as np


In [4]:


DATASET_NAME = 'repogle_k562_essential_raw'
dd = Catalogue.get_dataset_details(DATASET_NAME)
adata = sc.read(dd.path)


perts_original = [x for x in adata.obs[dd.pert_key].unique() if x != dd.control]
#random selection of 20 perts + control
perts = np.random.choice(adata.obs[dd.pert_key].unique(), 10, replace=False)
perts = set(perts).union(set([dd.control]))
adata = adata[adata.obs[dd.pert_key].isin(perts)]


adata.obs

,gem_group,gene,gene_id,transcript,gene_transcript,sgID_AB,mitopercent,UMI_count,z_gemgroup_UMI,core_scale_factor,core_adjusted_UMI_count,cell_type
cell_barcode,,,,,,,,,,,,
AAACCCAAGCGAGGAG-47,47,non-targeting,non-targeting,non-targeting,11059_non-targeting_non-targeting_non-targeting,non-targeting_02010|non-targeting_01147,0.092709,12221.0,1.096669,0.600264,20359.363281,k562
AAACCCAAGCGTCTGC-27,27,non-targeting,non-targeting,non-targeting,11020_non-targeting_non-targeting_non-targeting,non-targeting_01763|non-targeting_02345,0.087307,14638.0,0.760877,0.813253,17999.330078,k562
AAACCCAAGGAGGGTG-47,47,non-targeting,non-targeting,non-targeting,11097_non-targeting_non-targeting_non-targeting,non-targeting_02274|non-targeting_01075,0.144649,11241.0,0.813080,0.600264,18726.748047,k562
AAACCCAAGTACCCTA-20,20,non-targeting,non-targeting,non-targeting,11320_non-targeting_non-targeting_non-targeting,non-targeting_03721|non-targeting_02172,0.101813,20302.0,1.392648,1.030337,19704.224609,k562
AAACCCAAGTGTTCAC-3,3,non-targeting,non-targeting,non-targeting,10969_non-targeting_non-targeting_non-targeting,non-targeting_01395|non-targeting_02523,0.085352,16098.0,0.087551,1.112077,14475.623047,k562
...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTCGGCCTA-1,1,non-targeting,non-targeting,non-targeting,11187_non-targeting_non-targeting_non-targeting,non-targeting_02821|non-targeting_02347,0.121196,15446.0,1.213461,0.801973,19259.994141,k562
TTTGTTGGTCGTCTCT-6,6,non-targeting,non-targeting,non-targeting,11290_non-targeting_non-targeting_non-targeting,non-targeting_03530|non-targeting_01407,0.082511,8811.0,-1.069496,0.954127,9234.618164,k562
TTTGTTGGTGCTATTG-36,36,non-targeting,non-targeting,non-targeting,11248_non-targeting_non-targeting_non-targeting,non-targeting_03258|non-targeting_01564,0.072660,8409.0,-1.306466,1.011235,8315.576172,k562


In [5]:

adata.obs["condition"] = adata.obs[dd.pert_key]
adata.obs["cell_type"] = adata.obs[dd.cell_key]
perts = [p for p in adata.obs["condition"].unique() if p != dd.control]
adata.obs["condition"] = adata.obs["condition"].replace({dd.control:"ctrl"})
adata.obs["condition"] = adata.obs["condition"].replace({p:p+"+ctrl" for p in perts})
adata.var["gene_name"] = adata.var_names




In [6]:



print(f"Data loaded from {dd.path}")
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
print(f"Data normalized and log transformed")

device = 'cuda' if torch.cuda.is_available() else 'cpu'



Data loaded from /orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/K562_essential_raw_singlecell_01.h5ad
Data normalized and log transformed


In [7]:
pert_data_new = pertdata_new.PertData('./data_new') # specific saved folder


print(f"Saving data in ./data")

pert_data_new.new_data_process(dataset_name = DATASET_NAME, adata = adata, skip_calc_de=True) # specific dataset name and adata object
print(f"Data processed and saved in {pert_data_new.data_path}")
pert_data_new.prepare_split(split = 'no_test', seed = 1) # get data split with seed
print(f"Data split with seed 1")
pert_data_new.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader
print(f"Data loader prepared")


Downloading...
100%|██████████| 9.46M/9.46M [00:00<00:00, 11.8MiB/s]
Downloading...


Saving data in ./data
Starting new data processing for repogle_k562_essential_raw


100%|██████████| 559k/559k [00:00<00:00, 2.47MiB/s]


Creating 10691 cell graphs for ctrl: 0.42 seconds
create_cell_graph_dataset call duration: ctrl: 0.74 seconds
Creating 123 cell graphs for HIST1H2AE+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: HIST1H2AE+ctrl: 0.18 seconds
Creating 225 cell graphs for POLR2E+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: POLR2E+ctrl: 0.18 seconds
Creating 130 cell graphs for SNRPF+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: SNRPF+ctrl: 0.17 seconds
Creating 97 cell graphs for RFC2+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: RFC2+ctrl: 0.16 seconds
Creating 29 cell graphs for HSPE1+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: HSPE1+ctrl: 0.12 seconds
Creating 175 cell graphs for ATP6AP2+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: ATP6AP2+ctrl: 0.19 seconds
Creating 144 cell graphs for RPS5+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: RPS5+ctrl: 0.17 seconds
Creating 143 cell graphs for COX17+ctrl: 0

Creating new splits....
Saving new splits at ./data_new/repogle_k562_essential_raw/splits/repogle_k562_essential_raw_no_test_1_0.75.pkl
Done!
Creating dataloaders....
Done!


Creating 10 cell graphs for PPY+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: PPY+ctrl: 0.11 seconds
Done!
Data processed and saved in ./data_new
Data split with seed 1
Data loader prepared


In [8]:

gears_model = GEARS(pert_data_new, device = device, 
                        weight_bias_track = False, 
                        proj_name = 'pertnet_new', 
                        exp_name = 'pertnet_new')

gears_model.model_initialize(hidden_size = 64)

gears_model.tunable_parameters()




Downloading...
100%|██████████| 60.7M/60.7M [00:02<00:00, 21.9MiB/s]
Extracting tar file...
Done!


{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

In [11]:
next(iter(gears_model.dataloader["train_loader"]))

DataBatch(x=[160000, 1], y=[32, 5000], pert_idx=[32], de_idx=[32], pert=[32], batch=[160000], ptr=[33])

In [9]:

print(f"Training model")
gears_model.train(epochs = 1, lr = 1e-3)



Start Training...


Training model
Dataloader element: DataBatch(x=[274016, 1], y=[32, 8563], pert_idx=[32], de_idx=[32], pert=[32], batch=[274016], ptr=[33])


Epoch 1 Step 1 Train Loss: 0.4162
Epoch 1 Step 51 Train Loss: 0.4048
Epoch 1 Step 101 Train Loss: 0.5993
Epoch 1 Step 151 Train Loss: 0.4136
Epoch 1 Step 201 Train Loss: 0.9998
Epoch 1 Step 251 Train Loss: 0.2500
Epoch 1 Step 301 Train Loss: 0.5432
Epoch 1 Step 351 Train Loss: 0.4378
Epoch 1: Train Overall MSE: 0.0929 Validation Overall MSE: 0.0136. 
Train Top 20 DE MSE: 0.0892 Validation Top 20 DE MSE: 0.0104. 
Done!
Done! No test dataloader detected.


In [10]:


target = perts_original[0]
target 


'NAF1'

In [13]:

#GEARs returns bulk predictions, we transform those in single cell predictions
bulk_pred = gears_model.predict([[target]])[target]
bulk_pred


array([0.04402415, 0.07868292, 0.8854413 , ..., 0.27767277, 0.10376503,
       0.17194343], dtype=float32)

In [17]:
def distribute_shift(ctrl_cells, mean_shift):
    """
    Distribute the global per-gene difference (sum_diff[g]) across cells in proportion
    to the cell's existing counts for that gene. 
    """ 
    ctrl_cells = ctrl_cells.copy()
    sum_shift = (mean_shift * ctrl_cells.shape[0]).astype(int)

    n_cells, n_genes = ctrl_cells.shape


    #Its a matrix right now
    sum_shift = np.squeeze(np.array(sum_shift))

    #For each gene, distribute sum_diff[g] using a single multinomial draw
    for g in range(n_genes):
        diff = int(sum_shift[g])
        if diff == 0:
            continue  

        # Current counts for this gene across cells
        gene_counts = ctrl_cells[:, g].astype(np.float64)

        current_total = gene_counts.sum().astype(np.float64)
        

        # Probabilities ~ gene_counts / current_total
        p = gene_counts / current_total


        if diff > 0:
            # We want to add `diff` counts
            draws = np.random.multinomial(diff, p)  # shape: (n_cells,)
            
            ctrl_cells[:, g] = gene_counts + draws
        else:
            if current_total <= 0:
                continue

            # We want to remove `abs(diff)` counts
            amt_to_remove = abs(diff)

            to_remove = min(amt_to_remove, current_total)
            draws = np.random.multinomial(to_remove, p)
            # Subtract, then clamp
            updated = gene_counts - draws
            updated[updated < 0] = 0
            ctrl_cells[:, g] = updated

    return ctrl_cells


In [18]:

bulk_pred = np.array(bulk_pred)


control_cells = adata[(adata.obs[dd.pert_key] == dd.control) & (adata.obs[dd.cell_key] == 'k562')].X.toarray()


res = None

res = distribute_shift(control_cells, bulk_pred)

bulk_pred

array([0.04402415, 0.07868292, 0.8854413 , ..., 0.27767277, 0.10376503,
       0.17194343], dtype=float32)

In [19]:
res

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 5.0600405 , ..., 1.6643586 , 0.        ,
        1.6643586 ],
       [0.        , 2.8014312 , 1.8014312 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.97451407, 2.4585588 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.61097777, ..., 1.6109778 , 1.6109778 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.5291286 , 0.        ,
        2.5291286 ]], dtype=float32)